# 1) Create source sound collection

This notebook includes the code to create the collection of sounds that will later be used as source material for our audio mosaicing application. The collection of sounds is created by defining a number of queries to be performed using the Freesound API and concatenanting the results of each query. A number of metadata fields are stored for each sound in the collection and saved into a Pandas DataFrame object and CSV file in disk. For each sound in the collection, we also download an OGG preview and store it in disk.

This notebook uses the `freesound` Python package for interacting with the Freesound API. The source code for this package can be found here: https://github.com/mtg/freesound-python. In this repository you'll find a Python script with [examples](https://github.com/MTG/freesound-python/blob/master/examples.py) to learn how to interact with the API. Nevertheless, if you are further interested in the Freesound API, check the [API documentation](http://freesound.org/docs/api/) which provides more information.

**NOTE**: A Freesound API key is provided in this notebook, but you should make a Freesound account and get your own key. You can get a key here: https://freesound.org/apiv2/apply/

In [2]:
# Essentia
!pip install essentia
# Freesound-python
!pip install git+https://github.com/mtg/freesound-python.git
# Mount drive and cd to notebook folder
# from google.colab import drive
# drive.mount('/content/drive')


DEPRECATION: Loading egg at /Users/qinliu/anaconda3/lib/python3.11/site-packages/freesound_python-1.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
DEPRECATION: Loading egg at /Users/qinliu/anaconda3/lib/python3.11/site-packages/freesound_python-1.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Cloning https://github.com/mtg/freesound-python.git to /private/var/folders/vz/_sbkt1_95_jdlv4y0hb6db100000gn/T/pip-req-build-2375gdir
  Running command git clone --filter=blob:none --quiet https://github.com/mtg/freesound-python.git /private/var/folders/vz/_sbkt1_95_jdlv4y0hb6db100000gn/T/pip-req-build-2375gdir
  Resolved https://github.com/mtg/freesound-python.git to commit 5be99a3689d17303c01cb12

In [16]:
import os
import pandas as pd
import numpy as np
import sys
sys.path.append('/Users/qinliu/Postgraduate/Master/MTG-Audio_Signal_Processing/freesound-python/')  # directory were you installed the freesound-python repository
import freesound
from IPython.display import display

FREESOUND_API_KEY = 'qPiYCsYNsQK0FKgX0j5zJo06nuMKZmvNpnx9Rpqe'  # Please replace by your own Freesound API key
FILES_DIR = 'files'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME_BREAK = 'dataframe_break.csv'  # File where we'll store the metadata of our sounds collection
DATAFRAME_FILENAME_PIANO = 'dataframe_piano.csv'
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store

freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)
if not os.path.exists(FILES_DIR): os.mkdir(FILES_DIR)

In [7]:
# Define some util functions

def query_freesound(query, filter, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]

def retrieve_sound_preview(sound, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.previews.preview_hq_ogg.split('/')[-1])
    )

def make_pandas_record(sound):
    """Create a dictionary with the metadata that we want to store for each sound.
    """
    record = {key: sound.as_dict()[key] for key in FREESOUND_STORE_METADATA_FIELDS}
    del record['previews']  # Don't store previews dict in record
    record['freesound_id'] = record['id']  # Rename 'id' to 'freesound_id'
    del record['id']
    record['path'] = "files/" + sound.previews.preview_hq_ogg.split("/")[-1]  # Store path of downloaded file
    return record

# 1. Break Beat Sound Collection

In [17]:
# Build our collection of sounds

# Our collection of sounds is made by appending the results of a number of different queries to freesound
# The query terms, query filters and the number of results per query are all defined here.
# Information about how to define filters can be found in the Freesound API documentation: https://freesound.org/docs/api/resources_apiv2.html#request-parameters-text-search-parameters
freesound_queries = [
    # idea is for the zaps to be lower drum kicks
    # idea for snare to be glass smash
    {
        'query': 'bass',
        'filter': 'duration:[0 TO 1]',
        'num_results': 20, 
    },
    {
        'query': 'sub',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    }, 
    {
        'query': '303',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    },
    {
        'query': 'clap',
        'filter': 'duration:[0 TO 2]',
        'num_results': 20, 
    }
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME_BREAK)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME_BREAK))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 80 entries! dataframe_break.csv


,name,username,license,tags,freesound_id,path
0,Standup_Bass_Stop_Finger_F4.wav,pjcohen,http://creativecommons.org/publicdomain/zero/1.0/,"[Stereo, Bass, Instrument, Double, Plucked, St...",383560,files/383560_394391-hq.ogg
1,SID-ResBass-short-C#4.wav,supadoh,http://creativecommons.org/licenses/sampling+/...,"[bass, c64, sid]",51513,files/51513_610202-hq.ogg
2,BASS 0209.wav,zgump,http://creativecommons.org/publicdomain/zero/1.0/,[bass],83253,files/83253_377011-hq.ogg
3,SID-ResBass-short-A#1.wav,supadoh,http://creativecommons.org/licenses/sampling+/...,"[bass, c64, sid]",51498,files/51498_610202-hq.ogg
4,ElectroBass13.wav,gellski,http://creativecommons.org/licenses/by/3.0/,"[reversed, hard, bass, distortion]",288881,files/288881_1471115-hq.ogg
...,...,...,...,...,...,...
75,[Drums] Unsliced Live Claps 000,waveplaySFX,http://creativecommons.org/publicdomain/zero/1.0/,"[live, recording, recorded, sliced, record, un...",541279,files/541279_1676145-hq.ogg
76,Clap at One Church Corridor 4.wav,jonsept,http://creativecommons.org/publicdomain/zero/1.0/,"[atmosphere, room, echo, ambience, bang, atmos...",579138,files/579138_4185768-hq.ogg
77,clap.wav,dpsa,http://creativecommons.org/licenses/by/3.0/,"[sound, dpsa, clap]",341593,files/341593_5871007-hq.ogg
78,Clap 01,steeltowngaming,http://creativecommons.org/publicdomain/zero/1.0/,"[clapping, hands, slap, clap, hand]",537735,files/537735_11976230-hq.ogg


# 2. Piano Source Collection

In [18]:
freesound_queries = [
    # idea is for the zaps to be lower drum kicks
    # idea for snare to be glass smash
    {
        'query': 'JX-8P',
        'filter': None,
        'num_results': 800, 
    }
]

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], query['num_results']) for query in freesound_queries],[])

# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(sounds):
    print('Downloading sound with id {0} [{1}/{2}]'.format(sound.id, count + 1, len(sounds)))
    retrieve_sound_preview(sound, 'files/')
    
# Make a Pandas DataFrame with the metadata of our sound collection and save it
df =  pd.DataFrame([make_pandas_record(s) for s in sounds])
df.to_csv(DATAFRAME_FILENAME_PIANO)
print('Saved DataFrame with {0} entries! {1}'.format(len(df), DATAFRAME_FILENAME_PIANO))

# Show the contents of our DataFrame (the metadata of our source collection)
display(df)

Saved DataFrame with 150 entries! dataframe_piano.csv


,name,username,license,tags,freesound_id,path
0,Roland JX-8P - Slow Bass - D#3 (51D#2-30U1.aif),modularsamples,http://creativecommons.org/publicdomain/zero/1.0/,"[multisample, single-note, synthesizer, Roland...",309684,files/309684_2050105-hq.ogg
1,Roland JX-8P - Bonus Bass - F1 (30F#0-JP7Q.aif),modularsamples,http://creativecommons.org/publicdomain/zero/1.0/,"[multisample, single-note, synthesizer, Roland...",307802,files/307802_2050105-hq.ogg
2,Roland JX-8P - Sensitive Sync - C#2 (37C#1-30R...,modularsamples,http://creativecommons.org/publicdomain/zero/1.0/,"[multisample, single-note, synthesizer, Roland...",308950,files/308950_2050105-hq.ogg
3,Roland JX-8P - Sensitive Square - D3 (50D2-KRM...,modularsamples,http://creativecommons.org/publicdomain/zero/1.0/,"[multisample, single-note, synthesizer, Roland...",308829,files/308829_2050105-hq.ogg
4,Roland JX-8P - Proto Filter - E6 (88E5-0PN5.aif),modularsamples,http://creativecommons.org/publicdomain/zero/1.0/,"[multisample, single-note, synthesizer, Roland...",308411,files/308411_2050105-hq.ogg
...,...,...,...,...,...,...
145,Fireplace Burning,EminYILDIRIM,http://creativecommons.org/publicdomain/zero/1.0/,"[flames, burning, fire, stove, burn, flame, fi...",595244,files/595244_10912485-hq.ogg
146,Wierd Impact Designed,EminYILDIRIM,https://creativecommons.org/licenses/by/4.0/,"[Impact, Hit, game, Designed, audio, Boom]",668272,files/668272_10912485-hq.ogg
147,Forest Footsteps,EminYILDIRIM,http://creativecommons.org/publicdomain/zero/1.0/,"[walk, footsteps, forest, nature]",595874,files/595874_10912485-hq.ogg
148,electric loading ramp open close,Garuda1982,https://creativecommons.org/licenses/by/4.0/,"[noise, door, electric, sfx, industry, loading...",637009,files/637009_2061858-hq.ogg
